# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

%matplotlib inline

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv( '../output_data/cities.csv')
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kahului,20.8947,-156.4700,81.90,62,20,13.80,US,1630005224
1,Ushuaia,-54.8000,-68.3000,40.62,48,0,0.81,AR,1630005309
2,Ploemeur,47.7333,-3.4333,64.67,76,90,12.66,FR,1630005310
3,Punta Arenas,-53.1500,-70.9167,41.11,70,40,8.05,CL,1630005310
4,Carnarvon,-24.8667,113.6333,59.07,71,0,11.50,AU,1630005311


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
# Store latitude and longitude
locations = weather_df[[ 'Lat', 'Lng']]

# Store Humidity
humidity = weather_df[ 'Humidity'].astype( 'float')

In [5]:
fig = gmaps.figure(center = (0,0), zoom_level= 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = humidity, 
                                 dissipating = False, 
                                 max_intensity = 100, 
                                 point_radius = 3
                                )

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
optimal_df = weather_df.loc[( weather_df[ 'Wind Speed'] <= 15) & ( weather_df[ 'Cloudiness'] == 0) & \
                                   ( weather_df[ 'Max Temp'] >= 75) & ( weather_df[ 'Max Temp'] <= 85)].dropna()

optimal_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
147,Wana,32.2989,69.5725,83.19,32,0,4.21,PK,1630005369
148,Gazojak,41.1875,61.4036,83.70,23,0,5.61,TM,1630005370
188,Benghazi,32.1167,20.0667,81.30,69,0,9.37,LY,1630005160
246,Bahār,34.9072,48.4414,76.44,16,0,12.35,IR,1630005407
332,Arraial do Cabo,-22.9661,-42.0278,84.06,51,0,14.97,BR,1630005439


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = optimal_df.loc[:, [ 'City', 'Country', 'Lat', 'Lng']]

# Add an empty hotel column to the DataFrame
hotel_df['Hotel Name'] = ""

# Display to check
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
147,Wana,PK,32.2989,69.5725,
148,Gazojak,TM,41.1875,61.4036,
188,Benghazi,LY,32.1167,20.0667,
246,Bahār,IR,34.9072,48.4414,
332,Arraial do Cabo,BR,-22.9661,-42.0278,


In [8]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {'type': 'hotel',
          'keyword': 'hotel',
          'radius': 5000,
          'key': g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row[ 'Lat']
    lng = row[ 'Lng']
    city = row[ 'City']
    
    # Add keyword to params dict
    params[ 'location'] = f'{lat},{lng}'
      
    # Assemble url, make API request, and print request to screen
    response = requests.get(base_url, params= params).json()
    print( f'Retrieving Results for Index { index}: { city}...')
    
    # Save hotel to DataFrame
    try:
        print( f'Closest hotel in { city} is',response['results'][0][ 'name'])
        hotel_df.loc[ index, "Hotel Name"] = response['results'][0][ 'name']

    # If no hotel is found, print message that city is being skipped
    except (KeyError, IndexError):
        print( 'No hotel found... city skipped.')
    

Retrieving Results for Index 147: Wana...
No hotel found... city skipped.
Retrieving Results for Index 148: Gazojak...
No hotel found... city skipped.
Retrieving Results for Index 188: Benghazi...
No hotel found... city skipped.
Retrieving Results for Index 246: Bahār...
No hotel found... city skipped.
Retrieving Results for Index 332: Arraial do Cabo...
No hotel found... city skipped.
Retrieving Results for Index 334: Caravelas...
No hotel found... city skipped.
Retrieving Results for Index 337: Darnah...
No hotel found... city skipped.
Retrieving Results for Index 385: Porto Torres...
No hotel found... city skipped.
Retrieving Results for Index 405: Nurota...
No hotel found... city skipped.
Retrieving Results for Index 427: Mashhad...
No hotel found... city skipped.
Retrieving Results for Index 428: Goubellat...
No hotel found... city skipped.
Retrieving Results for Index 501: Goderich...
No hotel found... city skipped.
Retrieving Results for Index 534: Batman...
No hotel found... ci

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))